In [1]:
# LSTM for international airline passengers problem with memory
import time
import pandas as pd
import numpy as np
import matplotlib.font_manager
from sklearn import svm
import sys
from matplotlib import pyplot
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn import metrics
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation, Embedding
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import random
import shutil
import datetime

In [2]:
#convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [3]:
def encode_num_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()
    if sd is None:
        sd = df[name].std()
    df[name] = (df[name] - mean)/sd

In [4]:
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace = True)

In [5]:
def encode_text_index(df, name):
    le=preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

In [6]:
def to_xy(df, target):
    result=[]
    for x in df.columns:
        if x != target:
            result.append(x)
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '_iter_') else target_type
    if target_type in (np.int64, np.int32):
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        return df.df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)
    

In [7]:
def to_XY (dataset, target):
    result= []
    for x in dataset.columns:
        if x != target: 
            result.append (x)
    target_type = dataset[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    return dataset.as_matrix(result).astype(np.float32), dataset.as_matrix([target]).astype(np.float32)

In [8]:
df =  pd.read_csv("wf1.csv")
#power = df["PowerGeneration"]
df = df.drop(columns={'Time'})
df.head()

,ForecastingTime,AirPressure,Temperature,Humidity,WindSpeed100m,WindSpeed10m,WindDirectionZonal,WindDirectionMeridional,PowerGeneration
0,25,0.392238,0.406113,0.638134,0.625094,0.548931,0.820776,0.883540,0.506970
1,26,0.379920,0.405190,0.614377,0.628325,0.563099,0.812227,0.890531,0.579394
2,27,0.367603,0.404268,0.590621,0.631556,0.577266,0.803525,0.897331,0.524848
3,28,0.360061,0.401325,0.598596,0.629573,0.582233,0.791599,0.906165,0.543939
4,29,0.352519,0.398381,0.606571,0.627590,0.587199,0.779416,0.914640,0.620606


In [ ]:
encode_num_zscore(df, 'ForecastingTime')
encode_num_zscore(df, 'AirPressure')
encode_num_zscore(df, 'Temperature')
encode_num_zscore(df, 'Humidity')
encode_num_zscore(df, 'WindSpeed100m')
encode_num_zscore(df, 'WindSpeed10m')
encode_num_zscore(df, 'WindDirectionZonal')
encode_num_zscore(df, 'WindDirectionMeridional')
encode_num_zscore(df, 'WindDirectionMeridional')

outcomes = encode_text_index(df, 'PowerGeneration')
num_classes = len(outcomes)
df.dropna(inplace=True, axis=1)
df[0:5]

In [1]:
x, y = to_xy(df, 'PowerGeneration')
# choose a number of time steps
n_steps_in, n_steps_out = 8, 2574
n_features = 1
X = x.reshape((x.shape[0], x.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=50, verbose=1)
# demonstrate prediction
#x_input = array([70, 80, 90])
#x_input = x_input.reshape((1, n_steps_in, n_features))
#yhat = model.predict(x_input, verbose=0)
#print(yhat)

NameError: name 'to_xy' is not defined

In [ ]:
x, y = to_xy(df, 'PowerGeneration')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
batch_size = 64
learning_rate = 0.1
#create neural net
model = Sequential()
model.add(Dropout(0.5))
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(100, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(25, activation='relu'))
#model.add(Dense(1, kernel_initializer='uniform'))
model.add(Dense(y.shape[1], activation='sigmoid'))
opt = Adam(lr=0.0001)
#opt = SGD(lr=learning_rate, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-8, patience=5,verbose=1,mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True)
model.fit(x,y, validation_data=(x_test,y_test),batch_size=batch_size, callbacks= [ checkpointer], verbose=1,epochs=10)
model.load_weights('best_weights.hdf5')
#,

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test, axis=1)
score = metrics.accuracy_score(y_eval,pred)
#loss, accuracy = model.evaluate(np.array(x_test), np.array(y_test))
print("Validation score: {}".format(score))

In [ ]:
y_pred_tree = model.predict(x_test)
print("Accuracy: " +  str(score))
print(metrics.classification_report(y_eval, pred))
print("Confusion Matrix")
#display(pd.crosstab(y_test, y_pred_tree, rownames=['True'], colnames=['attack_cat'], margins=True))

In [ ]:
y_new = df['PowerGeneration']
x_new = df.drop(df.columns.to_series()['PowerGeneration':], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.25, random_state=42) 
x_train = np.array(x_train) 

x_test = np.array(x_test) 
LSTMtrain_X= x_train.reshape(x_train.shape[0], 1, 8) 
LSTMtest_X= x_test.reshape(x_test.shape[0], 1, 8)

In [ ]:
batch_size = 8
window = 1
num_features=8
learning_rate = 0.01
# 1. define the network
model2 = Sequential()
#model2.add(LSTM(80,input_dim=39, activation='relu'))  # try using a GRU instead, for fun
model2.add(LSTM(100, input_shape=(window, num_features), consume_less="mem", activation='relu'))
#model2.add(Dropout(0.2))
#model2.add(Dense(num_features, activation='softmax'))
#model2.add(LSTM(100, activation='relu'))
model2.add(Dense(1, activation='softmax'))
model2.add(Activation('linear'))
#optimizer = SGD(lr=learning_rate, decay=1e-4, momentum=0.9, nesterov=True)
model2.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,verbose=1,mode='auto')
#checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True)
model2.fit(LSTMtrain_X, y_train, nb_epoch=500, batch_size=batch_size, validation_data=(LSTMtest_X, y_test), callbacks=[checkpointer,csv_logger])
model2.save("lstm1layer_model.hdf5")
#,callbacks=[checkpointer,csv_logger]
#callbacks= [monitor, checkpointer]
loss, accuracy = model2.evaluate(LSTMtest_X, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
#,monitor

In [ ]:
import pickle
# save the model to disk
filename = 'LSTM_model.sav'
pickle.dump(model2, open(filename, 'wb'))
print("Model saved to file")

In [ ]:
#y_pred = model2.predict_classes(LSTMtest_X)
y_pred = model2.predict(LSTMtest_X)
y_pred= (np.round(y_pred)).astype(np.int64)
oss, accuracy = model2.evaluate(LSTMtest_X, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
print(metrics.classification_report(y_test, y_pred))